In [ ]:
!pip install torchtext==0.6

['Collecting torchtext==0.6',
 '  Downloading torchtext-0.6.0-py3-none-any.whl.metadata (6.3 kB)',
 'Requirement already satisfied: tqdm in /usr/local/lib/python3.11/dist-packages (from torchtext==0.6) (4.67.1)',
 'Requirement already satisfied: requests in /usr/local/lib/python3.11/dist-packages (from torchtext==0.6) (2.32.3)',
 'Requirement already satisfied: torch in /usr/local/lib/python3.11/dist-packages (from torchtext==0.6) (2.1.0)',
 'Requirement already satisfied: numpy in /usr/local/lib/python3.11/dist-packages (from torchtext==0.6) (1.26.4)',
 'Requirement already satisfied: six in /usr/local/lib/python3.11/dist-packages (from torchtext==0.6) (1.17.0)',
 'Requirement already satisfied: sentencepiece in /usr/local/lib/python3.11/dist-packages (from torchtext==0.6) (0.2.0)',
 'Requirement already satisfied: charset-normalizer<4,>=2 in /usr/local/lib/python3.11/dist-packages (from requests->torchtext==0.6) (3.4.2)',
 'Requirement already satisfied: idna<4,>=2.5 in /usr/local/li

In [ ]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 56.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import tensorflow_datasets as tfds
import tensorflow as tf

# Load the TED Talks translation dataset
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True, as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/incomplete.8HNH31_1.0.0/ted_hrlr_translate-val…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/incomplete.8HNH31_1.0.0/ted_hrlr_translate-tes…

Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [4]:
for ko, en in train_examples.take(1):
    print("Bo Dao Nha:", ko.numpy().decode('utf-8'))
    print("English:", en.numpy().decode('utf-8'))

Bo Dao Nha: e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
English: and when you improve searchability , you actually take away the one advantage of print , which is serendipity .


In [5]:
num_train = train_examples.reduce(0, lambda x, _: x + 1).numpy()
num_val = val_examples.reduce(0, lambda x, _: x + 1).numpy()
print("Train size:", num_train)
print("Validation size:", num_val)

Train size: 51785
Validation size: 1193


In [6]:
tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
        (en.numpy() for pt, en in train_examples), target_vocab_size=2**13)
tokenizer_pt = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
        (pt.numpy() for pt, en in train_examples), target_vocab_size=2**13)


In [7]:
max_pt_len = 0
max_en_len = 0

# Loop through the training examples
for pt, en in tfds.as_numpy(train_examples):
    pt_len = len(tokenizer_pt.encode(pt.decode('utf-8')))
    en_len = len(tokenizer_en.encode(en.decode('utf-8')))
    max_pt_len = max(max_pt_len, pt_len)
    max_en_len = max(max_en_len, en_len)

print(f"Max Portuguese sentence length: {max_pt_len}")
print(f"Max English sentence length: {max_en_len}")

Max Portuguese sentence length: 220
Max English sentence length: 201


In [9]:
def encoder(lang1, lang2):
    lang1 = [tokenizer_pt.vocab_size] + tokenizer_pt.encode(
        lang1.numpy()) + [tokenizer_pt.vocab_size+1]
    lang2 = [tokenizer_en.vocab_size] + tokenizer_en.encode(
        lang2.numpy()) + [tokenizer_en.vocab_size+1]
    return lang1, lang2
def tf_encoder(pt, en):
    result_pt, result_en = tf.py_function(encoder, [pt, en], [tf.int64, tf.int64])
    result_pt.set_shape([None])
    result_en.set_shape([None])
    return result_pt, result_en

In [10]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

train_dataset = train_examples.map(tf_encoder)
train_dataset = train_examples.map(tf_encoder).shuffle(BUFFER_SIZE).padded_batch(
    BATCH_SIZE,
    padded_shapes=([230], [230]),
    padding_values=(tf.constant(0, dtype=tf.int64), tf.constant(0, dtype=tf.int64))
)
val_dataset = val_examples.map(tf_encoder).padded_batch(
    BATCH_SIZE,
    padded_shapes=([230], [230]),
    padding_values=(tf.constant(0, dtype=tf.int64), tf.constant(0, dtype=tf.int64))
)


In [11]:
from keras.preprocessing import sequence
class Encoder (tf.keras.Model):
  def __init__(self, vocab_size, embed_dim):
    super(Encoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = embed_dim)
    self.lstm1 = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
    self.lstm2 = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
    self.lstm3 = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
    self.lstm4 = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
  def call(self,pt):
    embed = self.embedding(pt)
    output1, state_h1, state_c1 = self.lstm1(embed)
    output2, state_h2, state_c2 = self.lstm2(output1)
    output3, state_h3, state_c3 = self.lstm3(output2)
    output4, state_h4, state_c4 = self.lstm4(output3)
    return output1, state_h1, state_c1, output2, state_h2, state_c2, output3, state_h3, state_c3, output4, state_h4, state_c4
class Decoder (tf.keras.Model):
  def __init__(self, vocab_size, embed_dim):
    super(Decoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = embed_dim)
    self.lstm1 = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
    self.lstm2 = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
    self.lstm3 = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
    self.lstm4 = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
    self.fc = tf.keras.layers.Dense(vocab_size)
  def call (self, en, state_h1, state_c1,state_h2, state_c2, state_h3, state_c3,state_h4, state_c4  ):
    embed = self.embedding(en)
    output1, state_h1, state_c1 = self.lstm1(embed, initial_state= [state_h1, state_c1])
    output2, state_h2, state_c2 = self.lstm2(output1, initial_state= [state_h2, state_c2])
    output3, state_h3, state_c3 = self.lstm3(output2, initial_state= [state_h3, state_c3])
    output4, state_h4, state_c4 = self.lstm4(output3, initial_state= [state_h4, state_c4])
    output = self.fc(output4)
    return output




In [12]:
class seq2seq ():
  def __init__ (self, vocab_size_pt, vocab_size_en, embed_dim):
    self.encoder = Encoder(vocab_size_pt, embed_dim)
    self.decoder = Decoder(vocab_size_en, embed_dim)
    self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    self.optimizer = tf.keras.optimizers.Adam()
    self.epoch = 5
  @tf.function
  def train_step(self, pt_batch, en_batch):
      dec_input = en_batch[:, :-1]
      real = en_batch[:, 1:]

      with tf.GradientTape() as tape:
          enc_outputs = self.encoder(pt_batch)
          output1, state_h1, state_c1, output2, state_h2, state_c2, output3, state_h3, state_c3, output4, state_h4, state_c4 = enc_outputs
          predictions = self.decoder(dec_input, state_h1, state_c1, state_h2, state_c2,  state_h3, state_c3, state_h4, state_c4)

          loss = self.loss_function(real, predictions)  # ✅ use masked loss

      variables = self.encoder.trainable_variables + self.decoder.trainable_variables
      gradients = tape.gradient(loss, variables)
      self.optimizer.apply_gradients(zip(gradients, variables))
      return loss


  def loss_function(self, real, pred):
      mask = tf.math.logical_not(tf.math.equal(real, 0))
      loss_ = self.loss_object(real, pred)
      mask = tf.cast(mask, dtype=loss_.dtype)
      loss_ *= mask
      return tf.reduce_mean(loss_)
  def train(self, train_dataset, val_dataset, epochs):
      for epoch in range(epochs):
          total_loss = 0
          for (batch, (pt_batch, en_batch)) in enumerate(train_dataset):
              batch_loss = self.train_step(pt_batch, en_batch)
              total_loss += batch_loss

          print(f'Epoch {epoch + 1}, Loss: {total_loss.numpy():.4f}')
  def train(self, train_dataset, val_dataset, epochs):
    for epoch in range(epochs):
        total_loss = 0
        for (batch, (pt_batch, en_batch)) in enumerate(train_dataset):
            batch_loss = self.train_step(pt_batch, en_batch)
            total_loss += batch_loss

        print(f'Epoch {epoch + 1}, Loss: {total_loss.numpy():.4f}')





In [ ]:
for pt, en in train_dataset:
    print("PT shape:", pt.shape)
    print("EN shape:", en.shape)

PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)
PT shape: (64, 230)
EN shape: (64, 230)


In [ ]:
VOCAB_SIZE_PT = tokenizer_pt.vocab_size + 2
VOCAB_SIZE_EN = tokenizer_en.vocab_size + 2

EMBED_DIM = 256

model = seq2seq(VOCAB_SIZE_PT, VOCAB_SIZE_EN, EMBED_DIM)
model.train(train_dataset, val_dataset, epochs=2)